# Tema 2. Análisis de secuencias

## Análisis de calidad de secuencias cortas

El objetivo del primer ejercicio es mapear un conjunto de secuencias cortas a un genoma de referencia.

Las secuencias cortas provienen de un experimento de secuenciación de un aisalado de coronavirus. Se encuentran en los archivos `data/ERR4423464_1.fastq` y `data/ERR4423464_2.fastq`, que son archivos de texto con formato **fastq**. Hay dos porque se trata de lecturas emparejadas (*paired end*): cada fragmento original de DNA ha sido secuenciado primero desde un extremo y después desde el opuesto. Las dos lecturas pueden llegar a solaparse, si el fragmento original no era demasiado largo. Primero, échale un vistazo al archivo. Deberías poder observar algo así:

    @ERR4423464.1 M00289:305:000000000-J7B3T:1:1101:18085:1894 length=109
    CTGCTACACGCTGCGAAGCTCCCAATTTGTAATAAGAAAGCGTTCGTGATGTAGCAACAGTGATTTCTTTAGGC...
    +ERR4423464.1 M00289:305:000000000-J7B3T:1:1101:18085:1894 length=109
    BC@CCDE9F-@C7C+++6@CEFF8,CFFCFA9F9,,,,,,C:FFFCF@,C9F99C,6C8<C9,EFFFFEE9,,9...


Puedes aprender más sobre el formato **fastq** en [aquí](https://en.wikipedia.org/wiki/FASTQ_format). ¿Puedes identificar las diferentes partes del formato? ¿Para qué sirve la cuarta línea de cada registro?

A continuación utilizaremos el paquete *ShortReads* de R para comprovar la calidad y características de las secuencias. Este paquete ha sido creado por Martin Morgan, Michael Lawrence, Simon Anders, y distribuido con licencia *Artistic-2.0* en [Bioconductor](http://www.bioconductor.org/packages/release/bioc/html/ShortRead.html).

In [6]:
# Para poder usar funciones de análisis de secuencias cortas
# necesitamos cargar algun paquete específico, como "ShortReads":

library(ShortRead, quietly = FALSE)    # la opción "quietly=TRUE" evita mensajes innecesarios.
#library(Biostrings, quietly = FALSE)

In [7]:
# La funicón qa() aplica un análisis estándard a los archivos indicados,
# en este caso, los únicos dos archivos con extensión "fastq" en la carpeta
# "data":

ResumenCalidad <- qa('data', type = 'fastq', pattern = '*.fastq')

In [3]:
# Podemos ver el contenido del objeto "ResumenCalidad" así:

ResumenCalidad

In [4]:
# Así como acceder a sus elementos individuales:

ResumenCalidad[['readCounts']]

In [5]:
# Edita este bloque para mostrar otros elementos del "ResumenCalidad",
# como por ejemplo "baseCalls". Para mostrar sólo una pequeña parte
# de los elementos más grandes, puedes usar las funciones head() y tail():
head(ResumenCalidad[['baseQuality']])

In [6]:
# Por último, podemos generar un archivo html con toda la información
# generada por la función qa().
report(ResumenCalidad, dest = 'ResumenCalidad.html')

Después de experimentar con los resultados de la función *qa()* del paquete *ShortRead*, habrás observado que en algunos aspectos no se ajusta a nuestros datos. Por ejemplo, las secuencias en un archivo **fastq** no están alineadas, y por tanto *qa()* no ha podido informar sobre la calidad de los alineamientos. Además, un análisis estándard puede no responder a todas nuestras necesidades. Ejecutando el código a continuación generarás un histograma de las longitudes de las secuencias en los dos archivos **fastq**.

Como las longitudes de las secuencias no parecen estar registradas en el informe de *qa()*, es necesario leer de nuevo los datos originales. Para ello, disponemos de diferentes funciones. Como en este caso particular se trata de dos archivos relativamente pequeños, podemos usar la función *readFastq()*, que cargará en memoria la totalidad de las secuencias. Pero es más habitual, para la finalidad de comprobar la calidad de los datos, usar solo una fracción aleatoria de las secuencias. Así pues, usaremos la función *FastqSampler()* para extraer 100000 secuencias de cada archivo.

In [8]:
muestra1 <- yield(FastqSampler('data/ERR4423464_1.fastq', n = 100000))
muestra2 <- yield(FastqSampler('data/ERR4423464_2.fastq', n = 100000))

# Para mostrar las primeras secuencias de cada muestra:
head(sread(muestra1))

In [9]:
# Podemos mostrar también, sus calidades:
head(quality(muestra1))

In [10]:
# Y extraer directamente sus longitudes:
head(width(muestra1))

In [11]:
# Y para hacer un histograma, podemos usar la función hist():
hist(width(muestra1))

El análisis de la calidad de las secuencias obtenidas de un experimento de secuenciación es un paso fundamental en el que deben eliminarse las secuencias de baja calidad, o recortarse si en algun extremo la secuenciación a producido resultados pobres.

## Mapeo de lecturas cortas
El siguiente paso es normalmente el de identificar la posición de origen, en un genoma de referencia, de todas las secuencias cortas obtenidas por secuenciación. Existen muchos programas disponibles para ello. En el entorno de R podemos utilizar una implementación de Bowtie-2, llamada *Rbowtie2*, creada por Zheng Wei y Wei Zhang y disponible en [Bioconductor](https://bioconductor.org/packages/release/bioc/html/Rbowtie2.html) bajo licencia *GPL (>= 3)*.

El genoma de referencia es el del SARS-CoV-2, y se encuentra en el archivo `data/referencia.fna`. Éstas son sus primeras líneas:

    >NC_045512.2 Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome
    ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAA
    CGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAAC
    TAATTACTGTCGTTGACAGGACACGAGTAACTCGTCTATCTTCTGCAGGCTGCTTACGGTTTCGTCCGTG
    TTGCAGCCGATCATCAGCACATCTAGGTTTCGTCCGGGTGTGACCGAAAGGTAAGATGGAGAGCCTTGTC
    CCTGGTTTCAACGAGAAAACACACGTCCAACTCAGTTTGCCTGTTTTACAGGTTCGCGACGTGCTCGTAC
    GTGGCTTTGGAGACTCCGTGGAGGAGGTCTTATCAGAGGCACGTCAACATCTTAAAGATGGCACTTGTGG
    CTTAGTAGAAGTTGAAAAAGGCGTTTTGCCTCAACTTGAACAGCCCTATGTGTTCATCAAACGTTCGGAT

¿Reconoces el formato? Antes de *mapear* o alinear las secuencias cortas al genoma de referencia es necesario preparar (de hecho, *indexar*) el genoma de referencia.

In [2]:
library(Rbowtie2)